# Knowledge Graph Pipeline: Proof-of-Concept (ODKE+)

This notebook implements a production-grade, end-to-end pipeline for knowledge extraction, inspired by the ODKE+ paper: https://arxiv.org/pdf/2509.04696

It performs the following steps:
1.  **Crawl** target websites for unstructured text.
2.  **Extract** structured data using an LLM (Gemini) guided by a Pydantic ontology.
3.  **Ground** the extracted facts using a second, lightweight LLM to verify them against the source text.
4.  **Corroborate** the facts by checking for staleness (deactivating old data) and resolving conflicts with existing data.
5.  **Ingest** the "winning" facts into a Neo4j graph, updating the central node properties.
6.  **Infer** new relationships based on the *active* state of the graph.


In [1]:
# ==============================================================================
# Block 1: Setup
# ==============================================================================
!pip install -q neo4j beautifulsoup4 requests google-generativeai python-dotenv huggingface_hub


In [5]:
import os
# from google.colab import userdata
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List, Type, TypeVar, Generic
import enum
from datetime import datetime
import requests
from bs4 import BeautifulSoup
# from google import genai
import json
from neo4j import GraphDatabase, Session, Transaction

## Configuration

This block writes all static configuration to a `config.py` file. This separates our logic from our configuration, making the pipeline easier to manage.

Get your API Key here: https://neo4j.com and https://aistudio.google.com

It contains:
* API keys (add them to Colab Secrets).
* Target URLs for our crawler.
* The Corroborator's "Trust Score" settings for different source domains.

In [ ]:
# ==============================================================================
# Block 2: config.py
# ==============================================================================
%%writefile config.py
import os
from google.colab import userdata

# --- API Keys ---
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# --- Target URLs for webscraping ---
TARGET_URLS = [
    "https://www.vblh.de/privatkunden/geldanlage/sparbrief.html",
    "https://www.vblh.de/privatkunden/geldanlage/tagesgeld.html"
]

FILIAL_URLS = [
    "https://www.vblh.de/ueber-uns/filialen/bispingen.html",
    "https://www.vblh.de/ueber-uns/filialen/soltau.html"
]

# --- Corroborator Configuration ---
SOURCE_TRUST_SCORES = {
    "https://www.vblh.de/": 0.9,          # Trust scores for different types of sources
    "https://intern.vblh.de/": 0.95
}

def get_trust_score(url: str) -> float:
    """Findet den passenden Trust Score für eine gegebene URL."""
    for domain, score in SOURCE_TRUST_SCORES.items():
        if url.startswith(domain):
            return score
    return 0.5 # some standard-Score

Writing config.py


## The Data Ontology

This block defines our "Ontology"—the strict Pydantic schemas that guide the LLM extraction.

* `ProvableFact`: The core atomic unit. It forces the LLM to return both a `value` and the `evidence` snippet that proves it.
* `ProvenanceModel`: The metadata for each fact, including its source URL and trust score.
* `ExtractionPackage`: A generic container that bundles the `data` (our models) with its `metadata`.
* `KnowledgeGraphData` / `BranchData`: The top-level schemas the LLM must populate.

In [ ]:
# ==============================================================================
# Block 3: ontology.py
# ==============================================================================
%%writefile ontology.py
from pydantic import BaseModel, Field
from typing import Optional, List, TypeVar, Generic
import enum
from datetime import datetime

# --- Enums ---
class ProductTypeEnum(str, enum.Enum):
    """Defines the allowed product types."""
    INTEREST_PRODUCT = "InterestProduct"
    CHECKING_ACCOUNT = "CheckingAccount"
    SECURITY = "Security"

class RiskClassStrEnum(str, enum.Enum):
    """Defines the risk classes as string enums."""
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"

class RoleTypeEnum(str, enum.Enum):
    """Defines the allowed role types for employees."""
    ADVISOR = "Advisor"
    SERVICE = "Service"

# --- Atomic Fact Model ---
class ProvableFact(BaseModel):
    """
    A single, provable fact, consisting of the extracted value
    and the exact text snippet (evidence) that supports it.
    """
    value: str = Field(..., description="The extracted fact, e.g., 'Max Musterman' or 'Branch Buchholz'.")
    evidence: str = Field(..., description="The exact text snippet from the source that proves this fact.")

# --- Metadata Models ---
class ProvenanceModel(BaseModel):
    """
    Describes the origin (provenance) and timestamp (versioning)
    of a single extraction.
    """
    url: str
    retrieved_at: datetime = Field(default_factory=datetime.now)
    trust_score: float

T = TypeVar('T', bound=BaseModel)
class ExtractionPackage(BaseModel, Generic[T]):
    """
    A generic container that bundles the extracted payload (e.g., BranchData
    or KnowledgeGraphData) with its provenance metadata.
    This is the object passed to the ingestor.
    """
    metadata: ProvenanceModel
    data: T

class GrounderResponse(BaseModel):
    """The schema for the Grounder's simple boolean response."""
    is_grounded: bool

# --- Data Models (Ontology) ---
class EmployeeModel(BaseModel):
    """Describes a single employee."""
    name: ProvableFact
    email: Optional[ProvableFact] = None
    phone: Optional[ProvableFact] = None
    role_type: RoleTypeEnum = Field(..., description="Classify the employee's role as 'Advisor' or 'Service'.")

class BranchModel(BaseModel):
    """Describes a single bank branch."""
    name: ProvableFact
    address: Optional[ProvableFact] = None
    employees: List[EmployeeModel] = []

class BranchData(BaseModel):
    """The top-level schema for extracting branch information."""
    branch: BranchModel

class ConditionModel(BaseModel):
    """Describes a single financial condition."""
    type: Optional[ProvableFact] = None
    min_amount: Optional[int] = None
    max_amount: Optional[int] = None
    term_years: Optional[int] = None
    interest_rate: Optional[ProvableFact] = None

class FAQModel(BaseModel):
    """Describes a single Question-Answer pair."""
    question: ProvableFact
    answer: ProvableFact

class ProductModel(BaseModel):
    """Describes the core product."""
    name: ProvableFact
    description: Optional[ProvableFact] = None

class ProductTypeModel(BaseModel):
    """Describes the classified type of the product."""
    name: ProductTypeEnum

class RiskClassModel(BaseModel):
    """Describes the estimated risk class of the product."""
    risk_class: RiskClassStrEnum = Field(..., description="Estimate the product risk and select the appropriate class from '1' to '5'.")

    @property
    def class_as_integer(self) -> int:
        return int(self.risk_class.value)

class KnowledgeGraphData(BaseModel):
    """The top-level schema for extracting financial product information."""
    product: ProductModel
    product_type: ProductTypeModel
    risk_class: RiskClassModel
    conditions: Optional[List[ConditionModel]] = None
    faqs: Optional[List[FAQModel]] = None

Writing ontology.py


In [ ]:
from ontology import KnowledgeGraphData, BranchData
import json

print("--- JSON Schema for KnowledgeGraphData ---")
kg_schema = KnowledgeGraphData.model_json_schema()
print(json.dumps(kg_schema, indent=2))

print("\n--- JSON Schema for BranchData ---")
branch_schema = BranchData.model_json_schema()
print(json.dumps(branch_schema, indent=2))


--- JSON Schema for KnowledgeGraphData ---
{
  "$defs": {
    "ConditionModel": {
      "description": "Describes a single financial condition.",
      "properties": {
        "type": {
          "anyOf": [
            {
              "$ref": "#/$defs/ProvableFact"
            },
            {
              "type": "null"
            }
          ],
          "default": null
        },
        "min_amount": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Min Amount"
        },
        "max_amount": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Max Amount"
        },
        "term_years": {
          "anyOf": [
            {
              "type": "integer"
            },
            {


## The ETL Logic

This is the core of the application. It contains all the functions that perform the Extract, Transform, and Load (ETL) process, following the ODKE+ architecture.

1.  **Crawler (`get_webpage_content`):** Fetches and cleans text from a URL.
2.  **Extractor (`extract_structured_data`):** Uses Gemini Pro (LLM 1) to convert text into a structured Pydantic object based on our `ontology.py`.
3.  **Grounder (`is_fact_grounded`, `ground_package`):** Uses Gemini Flash (LLM 2) to verify that every single fact (`ProvableFact`) is supported by its evidence. Facts that fail are "nulled" (removed).
4.  **Ingestor & Corroborator (`_tx_...` functions):** A series of robust Neo4j transactions that:

    a.  **Check for Staleness:** Deactivates all old facts from the source URL (`is_active = false`).

    b.  **Corroborate Conflicts:** Checks for active facts from *other* sources.

    c.  **Score & Ingest:** Applies the "Freshness > Trust" algorithm to determine a "winner".

    d.  **Activate:** Writes the "winner's" data to the main node (e.g., `Product.interest_rate`) and sets its `FROM_SOURCE` relationship to `is_active = true`.
    
5.  **Inference (`create_inferred_relationships`):** Cleans up old inferred relationships and creates new ones *only* between currently active nodes.

In [ ]:
# ==============================================================================
# Block 4: pipeline.py
# ==============================================================================
%%writefile pipeline.py
import requests
from bs4 import BeautifulSoup
from google import genai
import json
from neo4j import GraphDatabase, Session, Transaction
from typing import Optional, List, Type
import enum
from datetime import datetime

# Import our custom modules
import config
from ontology import *

# --- Client Initialization ---
client = genai.Client(api_key=config.GOOGLE_API_KEY)

# ==============================================================================
# 4.1 CRAWLER
# ==============================================================================
def get_webpage_content(url: str) -> Optional[str]:
    """Fetches the visible text from a webpage."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        for script in soup(["script", "style"]): script.extract()
        text = soup.get_text(separator=" ")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching webpage {url}: {e}")
        return None

# Rewrite the crawler to use the NASDAS 100 sample data instead



# ==============================================================================
# 4.2 EXTRACTOR
# ==============================================================================
def extract_structured_data(text: str, schema_class: Type[BaseModel]) -> Optional[BaseModel]:
    """Extracts knowledge from text using the Gemini API."""
    prompt = f"""
    Extract all relevant information from the following text and populate the provided data schema.
    IMPORTANT: The schema requires a `ProvableFact` object for many facts.
    You must fill the `value` field (the fact) AND the `evidence` field (the text snippet from the source text
    that proves the fact).

    **Text to Analyze:**
    ---
    {text}
    ---
    """
    try:
        # Gemini 2.5 flash
        response = client.models.generate_content(
            model="gemini-2.5-pro",
            contents=prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": schema_class,
            }
        )
        if response.parsed is None:
            print("ERROR: The SDK could not parse the response into the Pydantic schema.")
            return None
        print(f"Model response for schema '{schema_class.__name__}' parsed successfully.")
        return response.parsed
    except Exception as e:
        print(f"An unexpected ERROR occurred during extraction: {e}")
        return None

# ==============================================================================
# 4.3 GROUNDER
# ==============================================================================
def is_fact_grounded(fact: str, evidence: str) -> bool:
    """Checks with a 'lightweight' LLM if a fact is supported by an evidence snippet."""
    prompt = f"""
    Verify if the following fact can be inferred from the provided text snippet.
    The fact must be explicitly mentioned or directly logically derivable.

    Fact to verify:
    "{fact}"

    Can this fact be derived from the following snippet?:
    "{evidence}"
    """
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": GrounderResponse,
            }
        )
        if response.parsed:
            return response.parsed.is_grounded
        else:
            print(f"GROUNDER WARNING: Could not parse response. Defaulting to 'False'.")
            return False
    except Exception as e:
        print(f"GROUNDER ERROR: {e}. Defaulting to 'False'.")
        return False

def _ground_model_recursive(model_instance: BaseModel):
    """Recursively iterates through a Pydantic model and nullifies any ungrounded facts."""
    if model_instance is None: return

    # Use .model_fields for Pydantic v2+
    for field_name, field_obj in model_instance.model_fields.items():
        field_value = getattr(model_instance, field_name)
        if field_value is None: continue

        if isinstance(field_value, ProvableFact):
            if not field_value.value or not field_value.evidence:
                print(f"--- ⚠️ GROUNDING SKIPPED: Empty value/evidence for {field_name}. Removing.")
                setattr(model_instance, field_name, None)
                continue

            is_grounded = is_fact_grounded(field_value.value, field_value.evidence)
            if not is_grounded:
                print(f"--- ❌ GROUNDING FAILED: Fact '{field_value.value}' (for field '{field_name}') will be removed.")
                setattr(model_instance, field_name, None)
            else:
                print(f"--- ✅ GROUNDING PASSED: Fact '{field_value.value}' (for field '{field_name}')")

        elif isinstance(field_value, BaseModel):
            _ground_model_recursive(field_value)
        elif isinstance(field_value, list):
            for item in field_value:
                if isinstance(item, BaseModel):
                    _ground_model_recursive(item)

def ground_package(package: ExtractionPackage) -> ExtractionPackage:
    """Takes an ExtractionPackage and validates all ProvableFact instances within it."""
    print("\n" + "="*30 + f"\n🔬 STARTING GROUNDING PROCESS for {package.metadata.url}\n" + "="*30)
    _ground_model_recursive(package.data)
    print(f"🔬 GROUNDING PROCESS for {package.metadata.url} COMPLETED.")
    return package

# ==============================================================================
# 4.4 INGESTOR & CORROBORATOR
# ==============================================================================
def get_node_props(model: BaseModel) -> dict:
    """Converts a Pydantic model into a flat dictionary of VALUES for Neo4j node properties."""
    props = {}
    if model is None: return props
    # Use .model_fields.items() for Pydantic v2+
    for field_name, field_info in model.model_fields.items():
        field_value = getattr(model, field_name)
        if isinstance(field_value, ProvableFact):
            if field_value.value is not None: props[field_name] = field_value.value
        elif isinstance(field_value, enum.Enum):
             props[field_name] = field_value.value
        elif isinstance(field_value, (str, int, float, bool)) or field_value is None:
            props[field_name] = field_value
    return props

def get_rel_props(model: BaseModel) -> dict:
    """Converts a Pydantic model into a flat dictionary of EVIDENCE snippets for Neo4j relationship properties."""
    props = {}
    if model is None: return props
    for field_name, field_info in model.model_fields.items():
        field_value = getattr(model, field_name)
        if isinstance(field_value, ProvableFact):
            if field_value.evidence is not None: props[f"{field_name}_evidence"] = field_value.evidence
    return props

def _tx_corroborate_and_ingest(
    tx: Transaction,
    node_label: str,
    node_key: str,
    node_key_value: str,
    new_node_props: dict,
    new_rel_props: dict,
    meta: ProvenanceModel
):
    """Executes the Corroborator logic (Model B) in a single transaction."""

    new_rel_props['retrieved_at'] = meta.retrieved_at
    new_rel_props['trust_score'] = meta.trust_score

    query_find_old = f"""
    MATCH (n:{node_label} {{{node_key}: $key_value}})-[r_alt:FROM_SOURCE {{is_active: true}}]->(q_alt:Source)
    WHERE q_alt.url <> $url
    RETURN r_alt.retrieved_at AS old_ts, r_alt.trust_score AS old_trust
    """
    result = tx.run(query_find_old, key_value=node_key_value, url=meta.url)
    old_fact = result.single()

    is_candidate_winner = False
    if not old_fact:
        is_candidate_winner = True
    else:
        if meta.retrieved_at > old_fact['old_ts']: is_candidate_winner = True
        elif meta.retrieved_at == old_fact['old_ts']:
            if meta.trust_score >= old_fact['old_trust']: is_candidate_winner = True
            else: is_candidate_winner = False
        else: is_candidate_winner = False

    tx.run(f"MERGE (q:Source {{url: $url}}) MERGE (n:{node_label} {{{node_key}: $key_value}})",
           url=meta.url, key_value=node_key_value)

    if is_candidate_winner:
        print(f"--- 🏆 CORROBORATOR: NEW wins for {node_key_value}")
        tx.run(f"""
        MATCH (n:{node_label} {{{node_key}: $key_value}})
        OPTIONAL MATCH (n)-[r_alt:FROM_SOURCE {{is_active: true}}]->()
        SET n = $node_props, r_alt.is_active = false
        """, key_value=node_key_value, node_props=new_node_props)

        new_rel_props['is_active'] = True
        tx.run(f"""
        MATCH (n:{node_label} {{{node_key}: $key_value}}) MATCH (q:Source {{url: $url}})
        MERGE (n)-[r_new:FROM_SOURCE]->(q) SET r_new = $rel_props
        """, key_value=node_key_value, url=meta.url, rel_props=new_rel_props)
    else:
        print(f"--- 🛡️ CORROBORATOR: OLD wins for {node_key_value}")
        new_rel_props['is_active'] = False
        tx.run(f"""
        MATCH (n:{node_label} {{{node_key}: $key_value}}) MATCH (q:Source {{url: $url}})
        MERGE (n)-[r_new:FROM_SOURCE]->(q) SET r_new = $rel_props
        """, key_value=node_key_value, url=meta.url, rel_props=new_rel_props)

def _tx_ingest_product_package(tx: Transaction, package: ExtractionPackage[KnowledgeGraphData]):
    """Executes the entire product ingestion in a single transaction."""
    data = package.data
    meta = package.metadata

    print(f"--- ⏳ STALENESS-CHECK: Deactivating old Product-facts from {meta.url}...")
    tx.run("MATCH (n:Product|Condition|FAQ)-[r:FROM_SOURCE {is_active: true}]->(q:Source {url: $url}) SET r.is_active = false", url=meta.url)

    if not data.product or not data.product.name: return
    product_name = data.product.name.value
    print(f"Processing Product: {product_name} from {meta.url}")
    product_node_props = get_node_props(data.product)
    product_node_props['name'] = product_name
    _tx_corroborate_and_ingest(tx, "Product", "name", product_name, product_node_props, get_rel_props(data.product), meta)

    if data.product_type and data.product_type.name:
        type_name = data.product_type.name.value
        tx.run("MATCH (p:Product {name: $p_name}) MERGE (pt:ProductType {name: $t_name}) MERGE (p)-[:HAS_PRODUCT_TYPE]->(pt)", p_name=product_name, t_name=type_name)

    if data.risk_class and data.risk_class.risk_class:
        class_value = data.risk_class.risk_class.value
        tx.run("MATCH (p:Product {name: $p_name}) MERGE (s:RiskClass {risk_class: $c_value}) MERGE (p)-[:HAS_RISK_CLASS]->(s)", p_name=product_name, c_value=class_value)

    if data.conditions:
        for condition in data.conditions:
            if condition is None or condition.interest_rate is None: continue
            key = f"{product_name}_{condition.min_amount}_{condition.term_years}"
            condition_node_props = get_node_props(condition)
            condition_node_props['key'] = key
            _tx_corroborate_and_ingest(tx, "Condition", "key", key, condition_node_props, get_rel_props(condition), meta)
            tx.run("MATCH (p:Product {name: $p_name}), (k:Condition {key: $key}) MERGE (p)-[:HAS_CONDITION]->(k)", p_name=product_name, key=key)

    if data.faqs:
        for faq in data.faqs:
            if faq is None or faq.question is None: continue
            question_value = faq.question.value
            faq_node_props = get_node_props(faq)
            faq_node_props['question'] = question_value
            _tx_corroborate_and_ingest(tx, "FAQ", "question", question_value, faq_node_props, get_rel_props(faq), meta)
            tx.run("MATCH (p:Product {name: $p_name}), (f:FAQ {question: $q_value}) MERGE (p)-[:HAS_FAQ]->(f)", p_name=product_name, q_value=question_value)

def _tx_ingest_branch_package(tx: Transaction, package: ExtractionPackage[BranchData]):
    """Executes the entire branch ingestion in a single transaction."""
    data = package.data
    meta = package.metadata

    print(f"--- ⏳ STALENESS-CHECK: Deactivating old Branch-facts from {meta.url}...")
    # KORREKTUR: This is NOT an f-string. Use single curly braces.
    tx.run("MATCH (n:Branch|Employee)-[r:FROM_SOURCE {is_active: true}]->(q:Source {url: $url}) SET r.is_active = false", url=meta.url)

    if not data.branch or not data.branch.name: return
    branch_name = data.branch.name.value
    print(f"Processing Branch: {branch_name} from {meta.url}")
    branch_node_props = get_node_props(data.branch)
    branch_node_props['name'] = branch_name
    _tx_corroborate_and_ingest(tx, "Branch", "name", branch_name, branch_node_props, get_rel_props(data.branch), meta)

    if data.branch.employees:
        for employee in data.branch.employees:
            if employee is None or employee.name is None: continue
            employee_name = employee.name.value
            print(f"-- Processing Employee: {employee_name}")
            employee_node_props = get_node_props(employee)
            employee_node_props['name'] = employee_name
            _tx_corroborate_and_ingest(tx, "Employee", "name", employee_name, employee_node_props, get_rel_props(employee), meta)
            tx.run("MATCH (m:Employee {name: $m_name}), (f:Branch {name: $f_name}) MERGE (m)-[:WORKS_IN]->(f)", m_name=employee_name, f_name=branch_name)
            if employee.role_type:
                role_type_name = employee.role_type.value
                tx.run("MATCH (m:Employee {name: $m_name}) MERGE (st:RoleType {name: $r_name}) MERGE (m)-[:HAS_ROLE_TYPE]->(st)", m_name=employee_name, r_name=role_type_name)

def ingest_product_package(package: ExtractionPackage[KnowledgeGraphData]):
    """Manager function: Writes a product package in a single transaction."""
    driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))
    with driver.session() as session:
        session.execute_write(_tx_ingest_product_package, package)
    print(f"Ingestion transaction for Product package completed.")

def ingest_branch_package(package: ExtractionPackage[BranchData]):
    """Manager function: Writes a branch package in a single transaction."""
    driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))
    with driver.session() as session:
        session.execute_write(_tx_ingest_branch_package, package)
    print(f"Ingestion transaction for Branch package completed.")

# ==============================================================================
# 4.5 INFERENCE
# ==============================================================================
def create_inferred_relationships():
    """Creates inferred relationships ONLY between active nodes."""
    driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))
    with driver.session() as session:
        print("\n" + "="*50 + "\nPHASE 2: CREATE INFERRED RELATIONSHIPS\n" + "="*50)

        print("Deleting all old :ADVISES_ON relationships...")
        session.run("MATCH ()-[r:ADVISES_ON]->() DETACH DELETE r")

        cypher_query = """
        MATCH (m:Employee)-[r_m:FROM_SOURCE]->() WHERE r_m.is_active = true
        MATCH (m)-[:HAS_ROLE_TYPE]->(:RoleType {name: 'Advisor'})
        MATCH (p:Product)-[r_p:FROM_SOURCE]->() WHERE r_p.is_active = true
        MATCH (p)-[:HAS_PRODUCT_TYPE]->(:ProductType {name: 'InterestProduct'})
        MERGE (m)-[r:ADVISES_ON]->(p)
        RETURN count(r) AS new_relationship_count
        """

        print("Creating new relationships between *active* Advisors and *active* InterestProducts...")
        result = session.run(cypher_query)
        summary = result.single()
        if summary:
            print(f"--> {summary['new_relationship_count']} new :ADVISES_ON relationships created.")
    driver.close()

# ==============================================================================
# HELPERS
# ==============================================================================
def clear_database():
    """Empties the entire Neo4j database."""
    print("Clearing the Neo4j database before starting...")
    driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))
    with driver.session() as s:
        s.run("MATCH (n) DETACH DELETE n")
    print("Database cleared.")

def ingest_fake_data():
    """
    Ingests a single FAKE condition to test the Corroborator logic.
    """
    print("\n" + "="*50 + "\n🔬 TEST: Ingesting FAKE Corroborator Data\n" + "="*50)

    # 1. Define some Fake Data
    fake_url = "https://intern.vblh.de/sparbrief"
    fake_time_str = "2025-10-02T15:24:42.019052" # set date in the future or past to test corroborator outcomes
    fake_key = "Volksbank Sparbrief_5000_6"
    fake_product_name = "Volksbank Sparbrief"

    # 2. Create Pydantic Models
    try:
        fake_time = datetime.fromisoformat(fake_time_str)
        fake_trust = config.get_trust_score(fake_url)

        fake_meta = ProvenanceModel(
            url=fake_url,
            retrieved_at=fake_time,
            trust_score=fake_trust
        )

        fake_condition = ConditionModel(
            type=ProvableFact(value="Savings Bond", evidence="Fake Entry"),
            min_amount=5000,
            max_amount=49999,
            term_years=6,
            interest_rate=ProvableFact(value="2.50%", evidence="Fake Entry: 2.50% (internal)")
        )

        # 3. Get Node/Rel Properties
        node_props = get_node_props(fake_condition)
        node_props['key'] = fake_key
        rel_props = get_rel_props(fake_condition)

        # 4. Open a DB session and call the Corroborator directly
        driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))
        with driver.session() as session:

            # Deactivate old facts from this FAKE source first
            # KORREKTUR: {{is_active: true}} -> {is_active: true}
            session.run("""
                MATCH (n:Condition)-[r:FROM_SOURCE {is_active: true}]->(q:Source {url: $url})
                SET r.is_active = false
            """, url=fake_url)

            # Call the Corroborator transaction
            session.execute_write(
                _tx_corroborate_and_ingest,
                "Condition",
                "key",
                fake_key,
                node_props,
                rel_props,
                fake_meta
            )

            # 5. Link the (now active) condition to the product
            session.run("""
                MATCH (p:Product {name: $product_name})
                MATCH (k:Condition {key: $key})
                MERGE (p)-[:HAS_CONDITION]->(k)
            """, product_name=fake_product_name, key=fake_key)

        print(f"🔬 TEST: Fake condition '{fake_key}' for source '{fake_url}' ingested successfully.")

    except Exception as e:
        print(f"🔬 TEST ERROR while ingesting fake data: {e}")

Writing pipeline.py


## The Orchestrator

This block imports all the logic from our other files and orchestrates the end-to-end pipeline.

The flow is:
1.  **Phase 0:** Clear the database.
2.  **Phase 1 (Real Data):**
    * Process Product URLs (Crawl, Extract, Ground, Ingest).
    * **Inject Fake Data** to test the Corroborator.
    * Process Branch URLs (Crawl, Extract, Ground, Ingest).
3.  **Phase 2 (Inference):** Create inferred relationships based on the final *active* state of the graph.

In [ ]:
# ==============================================================================
# Block 5: main.py
# ==============================================================================
%%writefile main.py
import config
from ontology import *
from pipeline import (
    get_webpage_content,
    extract_structured_data,
    ground_package,
    ingest_product_package,
    ingest_branch_package,
    create_inferred_relationships,
    clear_database,
    ingest_fake_data
)

def run_ingestion():
    """Executes the entire ingestion process."""

    # --- PHASE 0: SETUP ---
    clear_database()

    # --- PHASE 1: REAL DATA INGESTION ---
    print("\n" + "="*50 + "\nPHASE 1: STARTING REAL DATA INGESTION\n" + "="*50)

    # 1a. Process Product URLs
    for url in config.TARGET_URLS:
        print("\n" + "="*50 + f"\nProcessing PRODUCT URL: {url}\n" + "="*50)
        webpage_text = get_webpage_content(url)
        if webpage_text:
            # Step 1: LLM extracts the payload
            llm_data = extract_structured_data(text=webpage_text, schema_class=KnowledgeGraphData)

            if llm_data:
                # Step 2: Create metadata (Provenance & Versioning)
                provenance = ProvenanceModel(url=url, trust_score=config.get_trust_score(url))

                # Step 3: Bundle the extraction package
                package = ExtractionPackage[KnowledgeGraphData](metadata=provenance, data=llm_data)

                # --- STEP 3.5: GROUNDING ---
                grounded_package = ground_package(package)

                # Step 4: Pass the package to the Corroborator/Ingestor
                ingest_product_package(grounded_package) # Pass the FILTERED package

    # --- PHASE 1.5: FAKE DATA INJECTION (for Corroborator test) ---
    ingest_fake_data()

    # 1b. Process Branch URLs
    for url in config.FILIAL_URLS:
        print("\n" + "="*50 + f"\nProcessing BRANCH URL: {url}\n" + "="*50)
        webpage_text = get_webpage_content(url)
        if webpage_text:
            # Step 1: LLM extracts the payload
            llm_branch_data = extract_structured_data(text=webpage_text, schema_class=BranchData)

            if llm_branch_data:
                # Step 2: Create metadata
                provenance = ProvenanceModel(url=url, trust_score=config.get_trust_score(url))

                # Step 3: Bundle the package
                package = ExtractionPackage[BranchData](metadata=provenance, data=llm_branch_data)

                # --- STEP 3.5: GROUNDING ---
                grounded_package = ground_package(package)

                # Step 4: Pass the package to the Corroborator/Ingestor
                ingest_branch_package(grounded_package) # Pass the FILTERED package

    # --- PHASE 2: INFERENCE ---
    create_inferred_relationships()

    print("\n" + "="*50 + "\nINGESTION PROCESS COMPLETED.\n" + "="*50)

# This part allows the file to be run as a script
if __name__ == "__main__":
    run_ingestion()

Writing main.py


## Test Query & Validate

This block contains our test queries. It is designed to be run *after* `main.py`.

It validates the final state of the graph, proving our logic works:
* **Simple Queries (1-5, 9):** These mimic an application (or an LLM-powered search). They *only* query the active facts (e.g., `k.interest_rate` or `r.is_active = true`) and are simple and fast.
* **Debug Queries (6-8):** These are for us, the developers. They query the *entire* history (both active and inactive facts) to show that our audit trail and staleness-checks are working.

In [ ]:
# ==============================================================================
# Block 6: query.py
# ==============================================================================
%%writefile query.py
from neo4j import GraphDatabase
import config

def query_graph():
    """Runs comprehensive test queries against the graph to validate the entire data structure."""
    driver = GraphDatabase.driver(config.NEO4J_URI, auth=(config.NEO4J_USERNAME, config.NEO4J_PASSWORD))

    with driver.session() as session:
        print("\n" + "="*50 + "\nRUNNING GRAPH QUERIES\n" + "="*50)

        # --- Query 1 ---
        print("\n--- Query 1: Overview of all *active* products with type and risk class ---")
        query1 = """
        MATCH (p:Product)-[r_p:FROM_SOURCE]->() WHERE r_p.is_active = true
        MATCH (pt:ProductType)<-[:HAS_PRODUCT_TYPE]-(p)-[:HAS_RISK_CLASS]->(s:RiskClass)
        RETURN p.name AS Product, pt.name AS Type, s.risk_class AS Risk
        ORDER BY Risk, Type, Product
        """
        result1 = session.run(query1)
        for record in result1: print(f"- {record['Product']} (Type: {record['Type']}, Risk: {record['Risk']})")

        # --- Query 2 ---
        print("\n--- Query 2: Which *active* employees work in which *active* branch? ---")
        query2 = """
        MATCH (m:Employee)-[r_m:FROM_SOURCE]->() WHERE r_m.is_active = true
        MATCH (f:Branch)-[r_f:FROM_SOURCE]->() WHERE r_f.is_active = true
        MATCH (m)-[:WORKS_IN]->(f)
        RETURN f.name AS Branch, collect(DISTINCT m.name) AS Employees
        ORDER BY Branch
        """
        result2 = session.run(query2)
        print("(This query now filters out 'stale' employees)")
        for record in result2:
            print(f"Branch '{record['Branch']}':")
            for employee in record['Employees']: print(f"  - {employee}")

        # --- Query 3 ---
        print("\n--- Query 3: Which *active* advisors can advise on *active* interest products? ---")
        query3 = """
        MATCH (m:Employee)-[r_m:FROM_SOURCE]->() WHERE r_m.is_active = true
        MATCH (p:Product)-[r_p:FROM_SOURCE]->() WHERE r_p.is_active = true
        MATCH (m)-[:ADVISES_ON]->(p)
        RETURN m.name AS Advisor, p.name AS Product
        ORDER BY Advisor, Product
        """
        result3 = session.run(query3)
        for record in result3: print(f"- {record['Advisor']} can advise on '{record['Product']}'")

        # --- Query 4 ---
        print("\n--- Query 4: Who in Bispingen can *currently* help me with a secure 5-year investment? ---")
        query4 = """
        MATCH (p:Product)-[r_p:FROM_SOURCE]->() WHERE r_p.is_active = true
        MATCH (k:Condition)-[r_k:FROM_SOURCE]->() WHERE r_k.is_active = true
        MATCH (p)-[:HAS_CONDITION]->(k)
        WHERE k.min_amount <= 60000 AND (k.max_amount IS NULL OR k.max_amount >= 60000) AND k.term_years = 5
        WITH p
        MATCH (p)-[:HAS_RISK_CLASS]->(s:RiskClass)
        WHERE s.risk_class IN ['1', '2']
        MATCH (m:Employee)-[r_m:FROM_SOURCE]->() WHERE r_m.is_active = true
        MATCH (m)-[:ADVISES_ON]->(p)
        MATCH (m)-[:WORKS_IN]->(f:Branch)
        WHERE f.name CONTAINS 'Bispingen'
        RETURN DISTINCT m.name AS ContactPerson, m.email AS Email
        """
        result4 = session.run(query4)
        print("Possible *active* contact persons in the Bispingen branch:")
        for record in result4: print(f"- {record['ContactPerson']} (Email: {record['Email']})")

        # --- Query 5 ---
        print("\n--- Query 5: In which *active* branches does Martin Zado work...? ---")
        employee_name = "Martin Zado"
        query5 = """
        CYPHER 25
        MATCH (m:Employee {name: $name})
        LET branches = COLLECT {
            MATCH (m)-[:WORKS_IN]->(f:Branch)
            MATCH (f)-[r_f:FROM_SOURCE]->() WHERE r_f.is_active = true
            RETURN f.name
        }
        LET advised_products_sk1 = COLLECT {
            MATCH (m)-[:ADVISES_ON]->(p:Product)
            MATCH (p)-[r_p:FROM_SOURCE]->() WHERE r_p.is_active = true
            MATCH (p)-[:HAS_RISK_CLASS]->(s:RiskClass {risk_class: '1'})
            RETURN p.name
        }
        RETURN m.name AS Employee, m.email AS Email, m.phone AS Phone, branches, advised_products_sk1
        """
        try:
            result5 = session.run(query5, name=employee_name)
            record5 = result5.single()
            if not record5: print(f"Employee '{employee_name}' not found.")
            else:
                print(f"Details for: {record5['Employee']}")
                print(f"  - Email: {record5['Email']}, Phone: {record5['Phone']}")
                print(f"  - Works in *active* branches: {record5['branches']}")
                print(f"  - Advises on *active* products (SK1): {record5['advised_products_sk1']}")
        except Exception as e: print(f"ERROR during Query 5: {e}")

        # --- Query 6 ---
        print("\n--- Query 6 (Debug): Where does the 'Bispingen Branch' fact come from (all versions)? ---")
        query6 = """
        MATCH (f:Branch)-[r:FROM_SOURCE]->(q:Source)
        WHERE f.name CONTAINS 'Bispingen'
        RETURN f.name AS Fact, q.url AS Source, r.retrieved_at AS Timestamp, r.is_active AS Active
        ORDER BY r.retrieved_at DESC
        """
        result6 = session.run(query6)
        for record in result6: print(f"- Fact: '{record['Fact']}' @ {record['Timestamp']} (Source: {record['Source']}) [Active: {record['Active']}]")

        # --- Query 7 ---
        print("\n--- Query 7 (Debug): What facts were *ever* extracted from the savings bond page? ---")
        query7 = """
        MATCH (n)-[r:FROM_SOURCE]->(q:Source)
        WHERE q.url CONTAINS 'sparbrief.html'
        RETURN labels(n) AS Type, COALESCE(n.name, n.key, n.question) AS NameOrKey, r.retrieved_at AS Timestamp, r.is_active AS Active
        ORDER BY Timestamp DESC, Type, NameOrKey
        """
        result7 = session.run(query7)
        print(f"Facts from savings bond page (newest first):")
        for record in result7: print(f"- [{record['Type'][0]}] {record['NameOrKey']} (Retrieved: {record['Timestamp']}) [Active: {record['Active']}]")

        # --- Query 8 ---
        print("\n--- Query 8 (Debug): What is the *evidence* for a branch named Bispingen? ---")
        query8 = """
        MATCH (f:Branch)-[r:FROM_SOURCE]->(q:Source)
        WHERE f.name CONTAINS 'Bispingen' AND r.name_evidence IS NOT NULL
        RETURN f.name AS FactValue, r.name_evidence AS FactEvidence, q.url AS Source, r.retrieved_at AS Timestamp, r.is_active AS Active
        ORDER BY r.retrieved_at DESC
        LIMIT 1
        """
        result8 = session.run(query8)
        record8 = result8.single()
        if record8: print(f"- Fact (found): '{record8['FactValue']}' (Newest from {record8['Timestamp']}) [Active: {record8['Active']}]\n  Evidence: '{record8['FactEvidence']}'")
        else: print("No evidence found for a branch named 'Bispingen'.")

        # --- Query 9 ---
        print("\n--- Query 9: 'How much interest for 30,000€ for 5 years?' ---")
        investment_amount = 30000
        investment_years = 5
        query9 = """
        MATCH (p:Product)-[:HAS_CONDITION]->(k:Condition)
        MATCH (k)-[r_k:FROM_SOURCE]->()
        WHERE r_k.is_active = true

        AND k.min_amount <= $amount
        AND (k.max_amount IS NULL OR k.max_amount >= $amount)
        AND k.term_years = $years

        RETURN p.name AS Product, k.interest_rate AS InterestRate, k.type AS ConditionType
        """
        result9 = session.run(query9, amount=investment_amount, years=investment_years)
        print(f"Results for an *active* investment of {investment_amount}€ over {investment_years} years:")
        records9 = list(result9)
        if not records9: print("  -> No matching *active* conditions found.")
        else:
            for record in records9: print(f"  - Product: '{record['Product']}', Interest Rate: {record['InterestRate']} (Type: {record['ConditionType']})")

    driver.close()

# allow the file to be run as a script
if __name__ == "__main__":
    query_graph()

Writing query.py


# Test LLM as a judge

Recent research have show that LLM as a jusdge is a valid strategy for evaluation generative output of out model.

We use it as a way to evaluate the system out put of our models.

The model are ask to evaluate on:
* Precision
* Faithfulness
* Comprehensiveness
* Relevant


In [9]:
#%%writefile llm_judge.py

#%%writefile llm_judge.py
import html
import re
import json
import time
import pandas as pd
import requests
from typing import Optional, Dict, Any, List


# Prompt used as SYSTEM input for our LLM judge
JUDGE_PROMPT = """
You are an expert financial evaluator and LLM judge. Your role is to assess the quality of a system-generated answer to a user question based on four dimensions:

1. **Precision** – Is the answer clear, specific, and accurate?
2. **Faithfulness** – Does the answer remain factually correct and grounded in the source text?
3. **Comprehensiveness** – Does the answer fully capture the core information and context?
4. **Relevance** – Is the answer aligned with the main topic and intent of the user question?

### Instructions:
- Rate each dimension on a scale of **1 to 5** (1 = very poor, 5 = excellent).
- Provide a **final overall score** (average or weighted if needed).
- Explain your reasoning for each dimension clearly and concisely.
- Output your evaluation in **valid JSON format** as shown below.

### Output Format:
<ANSWER>
{
  "precision_score": <integer>,
  "faithfulness_score": <integer>,
  "comprehensiveness_score": <integer>,
  "relevance_score": <integer>,
  "overall_score": <integer>,
  "reasoning": {
    "precision": "<your explanation>",
    "faithfulness": "<your explanation>",
    "comprehensiveness": "<your explanation>",
    "relevance": "<your explanation>",
    "overall": "<summary of judgment>"
  }
}
</ANSWER>

### Additional Guidelines:
- Be **objective and consistent** in scoring.
- If information is missing or ambiguous, deduct points and explain why.
- Do not include any extra text outside the JSON block.
<ANSWER>{}</ANSWER>
""".strip()

TEST_ORIGIN_TEXT = """
Apple There's little question that Apple (NASDAQ: AAPL) has become one of the most successful companies in history. 
It was clear that CEO Jensen Huang had a knack for skating to where the puck was going -- recognizing technology trends on the fly and adapting Nvidia's processors and the accompanying software to meet that need. 
The company, which began as a local online auction site, has evolved into the largest e-commerce and payments ecosystem in Latin America, serving 18 countries in the region.
""".strip()

# Generated sample from Copilot based on the origin text for evaluating the output
TEST_TRIPLET = {
  "triples": [
    {
      "subject": "Apple",
      "predicate": "has become",
      "object": "one of the most successful companies in history",
      "evidence": "Apple There's little question that Apple (NASDAQ: AAPL) has become one of the most successful companies in history.",
      "subject_type": "Organization",
      "object_type": "Descriptor",
      "confidence": 0.95
    },
    {
      "subject": "Apple",
      "predicate": "ticker symbol",
      "object": "NASDAQ: AAPL",
      "evidence": "Apple (NASDAQ: AAPL)",
      "subject_type": "Organization",
      "object_type": "TickerSymbol",
      "confidence": 0.99
    },
    {
      "subject": "Jensen Huang",
      "predicate": "role",
      "object": "CEO",
      "evidence": "It was clear that CEO Jensen Huang had a knack...",
      "subject_type": "Person",
      "object_type": "Role",
      "confidence": 0.9
    },
    {
      "subject": "Jensen Huang",
      "predicate": "recognized",
      "object": "technology trends on the fly",
      "evidence": "recognizing technology trends on the fly",
      "subject_type": "Person",
      "object_type": "Concept",
      "confidence": 0.85
    },
    {
      "subject": "Jensen Huang",
      "predicate": "adapted",
      "object": "Nvidia's processors and software to meet technology needs",
      "evidence": "adapting Nvidia's processors and the accompanying software to meet that need.",
      "subject_type": "Person",
      "object_type": "Product/Software",
      "confidence": 0.88
    },
    {
      "subject": "Nvidia",
      "predicate": "has products",
      "object": "processors and accompanying software",
      "evidence": "adapting Nvidia's processors and the accompanying software",
      "subject_type": "Organization",
      "object_type": "Product/Software",
      "confidence": 0.8
    },
    {
      "subject": "Unnamed company",
      "predicate": "began as",
      "object": "local online auction site",
      "evidence": "The company, which began as a local online auction site,",
      "subject_type": "Organization",
      "object_type": "Activity",
      "confidence": 0.75
    },
    {
      "subject": "Unnamed company",
      "predicate": "evolved into",
      "object": "largest e-commerce and payments ecosystem in Latin America",
      "evidence": "has evolved into the largest e-commerce and payments ecosystem in Latin America,",
      "subject_type": "Organization",
      "object_type": "OrganizationDescriptor",
      "confidence": 0.82
    },
    {
      "subject": "Largest e-commerce and payments ecosystem in Latin America",
      "predicate": "serves",
      "object": "18 countries in the region",
      "evidence": "serving 18 countries in the region.",
      "subject_type": "OrganizationDescriptor",
      "object_type": "GeographicScope",
      "confidence": 0.8
    }
  ],
  "notes": [
    "The text refers to 'CEO Jensen Huang' in the context of Nvidia, implying the role at Nvidia.",
    "The final paragraph describes an unnamed company; if the source is MercadoLibre (commonly matching this description), its name is not explicitly stated in the provided text, so we retain 'Unnamed company' to avoid assumptions."
  ]
}


# -------------------------- LM Studio Client -------------------------- #
class LMStudioLLM:
    """
    Minimal, testable client for LM Studio's REST API with a callable interface.

    Usage:
        llm = LMStudioLLM(
            model="openai/gpt-oss-20b",
            base_url="http://localhost:1234",
            timeout=30,
            temperature=0.2,
            max_tokens=512,
        )
        output_text = llm(prompt_text)
    """

    def __init__(
        self,
        model: str,
        base_url: str = "http://localhost:1234",
        timeout: float = 30.0,
        temperature: float = 0.2,
        max_tokens: int = 512,
        session: Optional[requests.Session] = None,
        system_message: Optional[str] = None,
    ):
        self.model = model
        self.base_url = base_url.rstrip("/")
        self.timeout = timeout
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.session = session or requests.Session()
        self.system_message = system_message  # If None, you can pass system content per call

        self._chat_url = f"{self.base_url}/v1/chat/completions"

    def __call__(self, user_prompt: str, system_prompt: Optional[str] = None) -> str:
        """
        Call the LM Studio chat endpoint and return the assistant content as string.
        """
        headers = {"Content-Type": "application/json"}
        messages = []

        # Prefer explicitly provided system prompt; fall back to self.system_message
        sys_content = system_prompt if system_prompt is not None else self.system_message
        if sys_content:
            messages.append({"role": "system", "content": sys_content})

        messages.append({"role": "user", "content": user_prompt})

        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
        }

        resp = self.session.post(self._chat_url, headers=headers, data=json.dumps(payload), timeout=self.timeout)
        if resp.status_code >= 400:
            raise RuntimeError(f"LM Studio HTTP {resp.status_code}: {resp.text}")

        raw = resp.json()
        try:
            return raw["choices"][0]["message"]["content"]
        except (KeyError, IndexError, TypeError):
            return ""


# -------------------------- Judge Module -------------------------- #
class LLM_Judge:
    def __init__(self, llm_callable: LMStudioLLM):
        """
        Args:
            llm_callable: A callable that accepts (prompt: str, system_prompt?: str) and returns a string.
        """
        self.llm = llm_callable
        self.prompt = JUDGE_PROMPT
        self.columns = [
            "precision_score",
            "faithfulness_score",
            "comprehensiveness_score",
            "relevance_score",
            "overall_score",
            "reasoning",
        ]
        self.result = pd.DataFrame(columns=self.columns)


    def _extract_raw_output(self, raw_output: str) -> Optional[str]:
        """
        Extract JSON block between <ANSWER> ... </ANSWER>.
        - Unescapes HTML entities first.
        - Falls back to extracting the first JSON object if tags are missing.
        """
        if not raw_output:
            return None
    
        text = html.unescape(raw_output)
    
        # Primary: look for tags
        start = text.find("<ANSWER>")
        end = text.find("</ANSWER>")
        if start != -1 and end != -1 and end > start:
            return text[start + len("<ANSWER>"): end].strip()
    
        # Secondary: try to extract first JSON object via brace matching
        # This handles cases where the model omits the tags but returns pure JSON.
        first_brace = text.find("{")
        if first_brace != -1:
            depth = 0
            for i in range(first_brace, len(text)):
                if text[i] == "{":
                    depth += 1
                elif text[i] == "}":
                    depth -= 1
                    if depth == 0:
                        candidate = text[first_brace:i+1]
                        candidate = candidate.strip()
                        # Sanity check for JSON-like content
                        if candidate.startswith("{") and candidate.endswith("}"):
                            return candidate
    
        # Tertiary: regex-based attempt (less reliable but sometimes useful)
        m = re.search(r"\{(?:[^{}]|(?R))*\}", text)  # nested braces not fully supported in Python regex
        if m:
            return m.group(0).strip()
    
        return None


    def _build_eval_prompt(self, origin_text: str, system_answer_json: Dict[str, Any]) -> str:
        """
        Build the user message that provides context for the judge.
        """
        return (
            "User Question: Extract subject–predicate–object triples from the source text.\n\n"
            "Source Text:\n"
            f"{origin_text}\n\n"
            "System-Generated Answer (JSON):\n"
            f"{json.dumps(system_answer_json, ensure_ascii=False, indent=2)}\n\n"
            "Please evaluate the System-Generated Answer strictly based on the Source Text.\n"
            "Output ONLY the JSON inside <ANSWER>...</ANSWER> tags, per the system instructions."
        )

    def evaluate(self, origin_text: str, relevant_triplet: Dict[str, Any]) -> pd.DataFrame:
        """
        Evaluate the entire generated triple set once and store the JSON result.
        Returns the current results DataFrame.
        """
        user_prompt = self._build_eval_prompt(origin_text, relevant_triplet)
        raw_output = self.llm(user_prompt, system_prompt=self.prompt)
        answer_block = self._extract_raw_output(raw_output)

        if answer_block is None:
            raise ValueError("Could not find <ANSWER>...</ANSWER> block in model output.")

        try:
            evaluation = json.loads(answer_block)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error parsing JSON from model output: {e}\nRaw: {answer_block}")

        # Normalize reasoning into a single string for DataFrame or keep dict
        if "reasoning" in evaluation and isinstance(evaluation["reasoning"], dict):
            # Keep as dict, store in the 'reasoning' column
            row = {
                "precision_score": evaluation.get("precision_score"),
                "faithfulness_score": evaluation.get("faithfulness_score"),
                "comprehensiveness_score": evaluation.get("comprehensiveness_score"),
                "relevance_score": evaluation.get("relevance_score"),
                "overall_score": evaluation.get("overall_score"),
                "reasoning": evaluation["reasoning"],
            }
        else:
            # Fallback: place the entire evaluation in reasoning
            row = {
                "precision_score": evaluation.get("precision_score"),
                "faithfulness_score": evaluation.get("faithfulness_score"),
                "comprehensiveness_score": evaluation.get("comprehensiveness_score"),
                "relevance_score": evaluation.get("relevance_score"),
                "overall_score": evaluation.get("overall_score"),
                "reasoning": evaluation.get("reasoning"),
            }

        self.result = pd.concat([self.result, pd.DataFrame([row])], ignore_index=True)
        return self.result

    def evaluate_per_triple(self, origin_text: str, relevant_triplet: Dict[str, Any]) -> pd.DataFrame:
        """
        Optional: Evaluate each triple individually and append all results.

        Returns:
            DataFrame with one row per triple evaluation.
        """
        triples: List[Dict[str, Any]] = relevant_triplet.get("triples", [])
        if not triples:
            raise ValueError("No 'triples' found in relevant_triplet.")

        rows = []
        for i, triple in enumerate(triples, start=1):
            per_triple_answer = {"triple": triple}
            user_prompt = (
                "User Question: Evaluate the correctness of a single extracted triple against the source text.\n\n"
                "Source Text:\n"
                f"{origin_text}\n\n"
                "Triple Under Review (JSON):\n"
                f"{json.dumps(per_triple_answer, ensure_ascii=False, indent=2)}\n\n"
                "Please evaluate strictly based on the Source Text.\n"
                "Output ONLY the JSON inside <ANSWER>...</ANSWER> tags, per the system instructions."
            )

            raw_output = self.llm(user_prompt, system_prompt=self.prompt)
            answer_block = self._extract_raw_output(raw_output)
            if answer_block is None:
                # Skip but record a placeholder row
                rows.append({
                    "precision_score": None,
                    "faithfulness_score": None,
                    "comprehensiveness_score": None,
                    "relevance_score": None,
                    "overall_score": None,
                    "reasoning": {"overall": f"Missing <ANSWER> tags for triple #{i}."}
                })
                continue

            try:
                evaluation = json.loads(answer_block)
                rows.append({
                    "precision_score": evaluation.get("precision_score"),
                    "faithfulness_score": evaluation.get("faithfulness_score"),
                    "comprehensiveness_score": evaluation.get("comprehensiveness_score"),
                    "relevance_score": evaluation.get("relevance_score"),
                    "overall_score": evaluation.get("overall_score"),
                    "reasoning": evaluation.get("reasoning"),
                })
            except json.JSONDecodeError:
                rows.append({
                    "precision_score": None,
                    "faithfulness_score": None,
                    "comprehensiveness_score": None,
                    "relevance_score": None,
                    "overall_score": None,
                    "reasoning": {"overall": f"JSON parse error for triple #{i}."}
                })

        df = pd.DataFrame(rows)
        self.result = pd.concat([self.result, df], ignore_index=True)
        return self.result

    def return_results(self) -> pd.DataFrame:
        return self.result


# -------------------------- Example Usage -------------------------- #
if __name__ == "__main__":
    # 1) Instantiate LM Studio client (adjust model name to what you've loaded in LM Studio)
    llm = LMStudioLLM(
        model="openai/gpt-oss-20b",
        base_url="http://localhost:1234", # Replace with the host instance of LMStudio, this is the default
        temperature=0.1,
        max_tokens=512,
    )

    # 2) Create Judge
    judge = LLM_Judge(llm)

    # 3a) Evaluate the whole answer (triples JSON)
    try:
        results_df = judge.evaluate(TEST_ORIGIN_TEXT, TEST_TRIPLET)
        print("Whole-answer evaluation:")
        print(results_df.tail(1))
    except Exception as e:
        print("Evaluation error:", e)

    # 3b) (Optional) Evaluate per triple
    try:
        per_triple_df = judge.evaluate_per_triple(TEST_ORIGIN_TEXT, TEST_TRIPLET)
        print("\nPer-triple evaluations (appended):")
        print(per_triple_df.tail(len(TEST_TRIPLET["triples"])))
    except Exception as e:
        print("Per-triple evaluation error:", e)



Whole-answer evaluation:
  precision_score faithfulness_score comprehensiveness_score relevance_score  \
0               3                  2                       3               4   

  overall_score                                          reasoning  
0             3  {'precision': 'The triples are generally clear...  

Per-triple evaluations (appended):
  precision_score faithfulness_score comprehensiveness_score relevance_score  \
1               5                  5                       5               5   
2               5                  5                       4               5   
3               5                  5                       4               5   
4               5                  5                       4               5   
5               5                  5                       5               5   
6               5                  5                       4               5   
7               4                  5                       3               5   


## Run Pipeline & Query

This final block executes our application.
1.  `main.run_ingestion()`: Runs the complete ETL pipeline (Clear DB, Crawl, Extract, Ground, Ingest, Infer).
2.  `query.query_graph()`: Runs all test queries to validate the final state of the Knowledge Graph.

In [ ]:
# ==============================================================================
# Block 7: Run the ingstion and query tests
# ==============================================================================
import main
import query

main.run_ingestion()

query.query_graph()

SecretNotFoundError: Secret NEO4J_URI does not exist.